In [1]:
from datetime import datetime

import matplotlib.cm as cm
import numpy as np
import pandas as pd
import pydeck as pdk
import requests

pd.set_option("display.max_columns", 100)

In [2]:
def load(url):
    return requests.get(url).json()

In [3]:
info = load(
    "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json"
)
status = load(
    "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json"
)

In [4]:
df_info = pd.DataFrame(info["data"]["stations"]).set_index("station_id")

In [5]:
df_status = (
    pd.DataFrame(status["data"]["stations"])
    .set_index("station_id")
    .drop(
        columns=[
            "stationCode",
            "num_bikes_available",
            "numBikesAvailable",
            "numDocksAvailable",
        ]
    )
)
df_status["last_reported"] = (
    pd.to_datetime(df_status.last_reported, unit="s", utc=True)
    .dt.tz_convert("Europe/Paris")
    .dt.strftime("%Y-%m-%d %H:%M")
)

In [6]:
def count_available_mechanical_and_ebike_bikes(data):
    mechanical = data[0]["mechanical"]
    ebike = data[1]["ebike"]
    return mechanical, ebike

In [7]:
df_status[
    ["num_bikes_available_mechanical", "num_bikes_available_ebike"]
] = pd.DataFrame(
    df_status.num_bikes_available_types.apply(
        count_available_mechanical_and_ebike_bikes
    ).tolist(),
    index=df_status.index,
)

In [8]:
df = df_info.join(df_status)
df["available_rate"] = (
    (df.num_bikes_available_mechanical + df.num_bikes_available_ebike)
    / df.capacity
).fillna(0)
df["available_rate_format"] = (df.available_rate * 100).astype(int).astype(str)

In [9]:
df.head(10)

,name,lat,lon,capacity,stationCode,rental_methods,num_bikes_available_types,num_docks_available,is_installed,is_returning,is_renting,last_reported,num_bikes_available_mechanical,num_bikes_available_ebike,available_rate,available_rate_format
station_id,,,,,,,,,,,,,,,,
213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN,"[{'mechanical': 2}, {'ebike': 1}]",31,1,1,1,2023-05-25 09:12,2,1,0.085714,8
653222953,Mairie de Rosny-sous-Bois,48.871257,2.486581,30,31104,[CREDITCARD],"[{'mechanical': 8}, {'ebike': 16}]",4,1,1,1,2023-05-25 09:14,8,16,0.800000,80
36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD],"[{'mechanical': 1}, {'ebike': 2}]",18,1,1,1,2023-05-25 09:14,1,2,0.142857,14
37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN,"[{'mechanical': 0}, {'ebike': 0}]",28,1,1,1,2023-05-25 09:12,0,0,0.000000,0
17278902806,Rouget de L'isle - Watteau,48.778193,2.396302,0,44015,NaN,"[{'mechanical': 0}, {'ebike': 0}]",0,0,0,0,2023-02-09 10:35,0,0,0.000000,0
251039991,Cassini - Denfert-Rochereau,48.837526,2.336035,25,14111,[CREDITCARD],"[{'mechanical': 2}, {'ebike': 1}]",21,1,1,1,2023-05-25 09:12,2,1,0.120000,12
85002689,Jourdan - Stade Charléty,48.819428,2.343335,60,14014,[CREDITCARD],"[{'mechanical': 17}, {'ebike': 7}]",32,1,1,1,2023-05-25 09:15,17,7,0.400000,40
2515829865,Basilique,48.936269,2.358867,22,32017,[CREDITCARD],"[{'mechanical': 3}, {'ebike': 2}]",16,1,1,1,2023-05-25 09:14,3,2,0.227273,22
516709288,Charonne - Robert et Sonia Delaunay,48.855908,2.392571,20,11104,NaN,"[{'mechanical': 1}, {'ebike': 2}]",16,1,1,1,2023-05-25 09:14,1,2,0.150000,15


In [10]:
def to_rgb(x, colormap):
    return np.around((np.array(colormap(x)) * 255)).tolist()


RED_OVER = (np.around(cm.Reds.get_over() * 255)).tolist()

df["color"] = df.available_rate.apply(lambda x: to_rgb(x, cm.RdYlGn))
df["color_ebike"] = (df.num_bikes_available_ebike / 10).apply(
    lambda x: to_rgb(x, cm.winter_r) if x > 0 else RED_OVER
)

df["color_mechanical"] = (df.num_bikes_available_mechanical / 10).apply(
    lambda x: to_rgb(x, cm.summer_r) if x > 0 else RED_OVER
)

In [11]:
PARIS_LATITUDE = 48.856614
PARIS_LONGITUDE = 2.3522219

WHAT_WE_WANT_TO_SEE = "color_ebike"  # "color_ebike", "color_mechanical" or "color" (for all bikes)


map_style = pdk.map_styles.DARK
initial_view_state = pdk.ViewState(
    latitude=PARIS_LATITUDE,
    longitude=PARIS_LONGITUDE,
    zoom=11.25,
)
layer = pdk.Layer(
    "ScatterplotLayer",
    data=df[df.is_installed == 1],  # drop disconnected velib stations
    get_position="[lon, lat]",
    get_fill_color=WHAT_WE_WANT_TO_SEE,
    get_radius=30,
    pickable=True,
    filled=True,
)
layers = [layer]
pdk_obj = pdk.Deck(
    map_style=map_style,
    initial_view_state=initial_view_state,
    layers=layers,
    tooltip={
        "text": """{name}
            Mechanical bikes: {num_bikes_available_mechanical}
            Ebikes : {num_bikes_available_ebike}
            Availability rate: {available_rate_format}%
            Last update: {last_reported}
            """,
    },
)

In [12]:
pdk_obj.to_html(f"velib_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.html")